In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data18 = data.loc[data['Year']== '2018']
data18 = data18.loc[data18['Genre1']== 'Folk/Country']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
10,11,The Lillywhite Sessions,Ryley Walker,"’s music is a stormy, searching amalgam of pro...",Dead Oceans,7.8,November,26,2018,Folk/Country,NaN,NaN,NaN
130,131,Ghost Forests,"Meg Baird,Mary Lattimore",listen for the twangs. the little disruptions ...,Three Lobed,7.4,November,24,2018,Folk/Country,Experimental,NaN,NaN
176,178,Car Wheels on a Gravel Road,Lucinda Williams,"when nothing but time can still the pain, a , ...",Mercury,9.5,October,28,2018,Folk/Country,NaN,NaN,NaN
199,201,First Collection 2006 – 2009,Fleet Foxes,"’ entrée to the greater world—2008’s , ep and ...",Sub Pop,8.1,November,10,2018,Folk/Country,NaN,NaN,NaN
212,214,This Is My Dinner,Sun Kil Moon,"someday soon, , might write a gorgeous tune ab...",Caldo Verde,2.8,November,14,2018,Folk/Country,NaN,NaN,NaN
214,216,Interstate Gospel,Pistol Annies,"whenever i think about ,, i think, invariably,...",RCA Records Nashville,8.0,November,5,2018,Folk/Country,NaN,NaN,NaN
239,241,If Only There Was a River,Anna St. Louis,"before her debut album, , flipped through the ...","Woodsist,Mare",7.5,October,30,2018,Folk/Country,NaN,NaN,NaN
17334,19622,Follow Them True,Stick in the Wheel,london’s stick in the wheel live up to their n...,From Here,7.7,February,1,2018,Folk/Country,NaN,NaN,NaN
17368,19656,Ruins,First Aid Kit,"as far back as their 2010 debut album, ,, , di...",Columbia,6.4,January,19,2018,Folk/Country,NaN,NaN,NaN
17456,19746,No Fool Like an Old Fool,Caroline Says,a friend recently admitted that she hadn’t lis...,Western Vinyl,7.5,March,21,2018,Folk/Country,NaN,NaN,NaN


In [5]:
data.Genre1.unique()

array(['Pop/R&B', 'Rock', 'Experimental', 'Rap', 'Electronic',
       'Folk/Country', 'Global', 'Metal', 'Jazz'], dtype=object)

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['music', 'is', 'stormy', 'searching', 'amalgam', 'of', 'proggy', 'british', 'folk', 'primitive', 'guitar', 'free', 'improv', 'and', 'chicago', 'jazz', 'rock', 'he', 'draws', 'from', 'deep', 'well', 'of', 'serious', 'record', 'influences', 'the', 'stuff', 'you', 'discover', 'once', 'you', 've', 'burned', 'through', 'the', 'standard', 'canon', 'but', 'to', 'get', 'there', 'you', 've', 'got', 'to', 'start', 'somewhere', 'and', 'for', 'walker', 'the', 'journey', 'began', 'with', 'single', 'see', 'walker', 'wasn', 'raised', 'on', 'or', 'the', 'he', 'came', 'up', 'fan', 'perhaps', 'working', 'out', 'the', 'changes', 'to', 'in', 'the', 'space', 'between', 'bongloads', 'dmb', 'musically', 'omnivorous', 'minded', 'jazz', 'folk', 'fusion', 'put', 'walker', 'on', 'his', 'own', 'roundabout', 'path', 'and', 'on', 'his', 'lp', 'of', 'full', 'album', 'reimagining', 'of', 'dmb', 'largely', 'lost', 'turn', 'of', 'the', 'millennium', 'album', 'he', 'finds', 'his', 'way', 'back', 'to', 'the', 'source',

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['music', 'is', 'stormy', 'searching', 'amalgam', 'of', 'proggy', 'british', 'folk', 'primitive', 'guitar', 'free', 'improv', 'and', 'chicago', 'jazz', 'rock', 'he', 'draws', 'from', 'deep', 'well', 'of', 'serious', 'record', 'influences', 'the', 'stuff', 'you', 'discover', 'once', 'you', 've', 'burned', 'through', 'the', 'standard', 'canon', 'but', 'to', 'get', 'there', 'you', 've', 'got', 'to', 'start', 'somewhere', 'and', 'for', 'walker', 'the', 'journey', 'began', 'with', 'single', 'see', 'walker', 'wasn', 'raised', 'on', 'or', 'the', 'he', 'came', 'up', 'fan', 'perhaps', 'working', 'out', 'the', 'changes', 'to', 'in', 'the', 'space', 'between', 'bongloads', 'dmb', 'musically', 'omnivorous', 'minded', 'jazz', 'folk', 'fusion', 'put', 'walker', 'on', 'his', 'own', 'roundabout', 'path', 'and', 'on', 'his', 'lp', 'of', 'full', 'album', 'reimagining', 'of', 'dmb', 'largely', 'lost', 'turn', 'of', 'the', 'millennium', 'album', 'he', 'finds', 'his', 'way', 'back', 'to', 'the', 'source', 

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['music', 'stormy', 'search', 'amalgam', 'proggy', 'british', 'folk', 'primitive', 'guitar', 'free', 'improv', 'chicago', 'jazz', 'rock', 'draw', 'deep', 'well', 'serious', 'record', 'influence', 'stuff', 'discover', 'burn', 'standard', 'canon', 'get', 'get', 'start', 'somewhere', 'journey', 'begin', 'single', 'see', 'walker', 'raise', 'come', 'fan', 'perhaps', 'work', 'change', 'space', 'bongload', 'dmb', 'musically', 'omnivorous', 'minded', 'jazz', 'folk', 'fusion', 'put', 'roundabout', 'path', 'lp', 'full', 'album', 'reimagin', 'dmb', 'largely', 'lose', 'turn', 'millennium', 'album', 'find', 'way', 'back', 'source', 'dmb', 'almost', 'certainly', 'widely', 'hear', 'full', 'length', 'bootleg', 'century', 'late', 'company', 'begin', 'work', 'follow', 'dense', 'daunting', 'longtime', 'producer', 'song', 'depressed', 'point', 'maudlin', 'make', 'air', 'studio', 'oppressive', 'week', 'stretch', 'month', 'frazzled', 'band', 'reach', 'detente', 'label', 'would', 'fly', 'meet', 'producer', 

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

ValueError: cannot compute LDA over an empty collection (no terms)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis